In [1]:
import pandas as pd
import numpy as np
import math
import gc

from sklearn.metrics import mean_squared_error

In [2]:
train = pd.read_csv("train_subset_2.csv")

In [3]:
# train = pd.read_csv("test.csv")

In [4]:
# train = test

In [5]:
attr = pd.read_csv("attr.csv")

In [6]:
train_2 = pd.merge(train, attr, on=['ego_id', 'u']).merge(attr, right_on=['ego_id', 'u'], left_on=['ego_id', 'v'])

In [7]:
del attr 
gc.collect()

0

In [8]:

train_2 = train_2.drop_duplicates()

In [9]:
def dis(x,y):
    return abs(x-y) if x > 0 and y > 0 else -1

In [11]:
def tmp(x, y):
    if x == -1 or y == -1:
        return -1
    return abs(x - y) 


train_2["city_dist"] = list(map(tmp, train_2["city_id_x"], train_2["city_id_y"]))
train_2["university_dist"] = list(map(tmp, train_2["university_x"], train_2["university_y"]))
train_2["sex"] = (train_2["sex_x"] + train_2["sex_y"]) % 2
train_2["school_dist"] = list(map(tmp, train_2["school_y"], train_2["school_x"]))
train_2["age_dist"] = abs(train_2["age_x"] - train_2["age_y"])
train_2["age_mean"] = (train_2["age_x"] + train_2["age_y"]) / 2



In [12]:
def sex_cat(x, y):
    if  x == 1 and y == 1:
        return 1
    elif x == 2 and y == 2:
        return 2
    else:
        return 0
    
def is_the_same(x, y):
    if  x == y and x != -1:
        return 2
    elif x != -1 or y != -1:
        return 0
    else:
        return 1
    
def age_eps(x):
    if x <= 8:
        return 1
    else: 
        return 0


train_2['sex2'] = list(map(sex_cat, train_2["sex_x"], train_2["sex_y"]))
train_2["city"] = list(map(is_the_same, train_2["city_id_x"], train_2["city_id_y"]))
train_2["university"] = list(map(is_the_same, train_2["university_x"], train_2["university_y"]))
train_2["school"] = list(map(is_the_same, train_2["school_y"], train_2["school_x"]))
train_2["gen"] = list(map(age_eps, train_2["age_dist"]))

In [13]:
train_3 = train_2.drop(["city_id_x", "city_id_y", "university_x", "university_y", "sex_x", "sex_y", "school_x", "school_y", "age_x", "age_y", "u_y"], axis=1)

In [14]:
del train_2
gc.collect()

0

In [15]:
train_3.rename(columns= {"u_x": "u"} , inplace=True)


In [16]:
grouped_data1 = train_3.groupby(['ego_id', 'u']).agg({'v':'count'}).reset_index().rename(columns = {'v':'v_count'})

In [17]:
grouped_data = train_3.groupby(['ego_id', 'v']).agg({'u':'count'}).reset_index().rename(columns = {'u':'u_count'})

In [18]:
grouped_data = pd.merge(grouped_data1, grouped_data, left_on = ['ego_id', 'u'], right_on = ['ego_id', 'v'])

In [19]:
grouped_data['freinds'] = grouped_data['v_count']+grouped_data['u_count']

In [20]:
df = pd.merge(train_3, grouped_data[['ego_id', 'u', 'freinds']], on = ['ego_id', 'u'])
df = pd.merge(df, grouped_data[['ego_id', 'v', 'freinds']], on = ['ego_id', 'v'])
del train_3, grouped_data, grouped_data1
gc.collect()
df = df.rename(columns = {'freinds_x':'freinds_u', 'freinds_y':'freinds_v'})

In [21]:
df['u_v'] = (1 + df['x2'])*df['freinds_u'] + (1 + df['x3'])*df['freinds_v']

In [22]:
df

,ego_id,u,v,t,x1,x2,x3,city_dist,university_dist,sex,...,age_dist,age_mean,sex2,city,university,school,gen,freinds_u,freinds_v,u_v
0,13,26,20,256.7,6.667426e-09,0.000000,1.0,0,-1,0,...,2,29.0,2,2,1,0,1,25,29,83.000000
1,13,0,20,144.7,1.536793e-04,0.000000,1.0,0,-1,0,...,5,32.5,2,2,0,2,1,220,29,278.000000
2,13,148,20,NaN,0.000000e+00,0.000000,1.0,0,-1,0,...,19,39.5,2,2,0,2,0,35,29,93.000000
3,13,2,20,79.1,6.092806e-07,0.000000,0.0,18712486,-1,0,...,5,32.5,2,0,0,0,1,30,29,59.000000
4,13,17,20,338.1,1.138490e-08,0.000000,1.0,0,-1,0,...,8,34.0,2,2,0,2,1,39,29,97.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26005982,1709396984692,12,2,9.5,7.487368e-01,0.000000,0.0,-1,-1,1,...,1,15.5,0,0,1,0,1,7,12,19.000000
26005983,1709396984692,15,2,7.5,1.835345e+00,2.484907,0.0,-1,-1,0,...,4,18.0,2,0,1,0,1,9,12,43.364160
26005984,1709396984692,0,18,NaN,0.000000e+00,2.484907,0.0,-1,-1,1,...,4,21.0,0,1,1,1,1,23,2,82.152853
26005985,1709396984692,0,10,0.9,1.993434e+00,2.302585,0.0,-1,-1,1,...,3,17.5,0,1,1,1,1,23,6,81.959457


In [23]:
df.to_csv("train_fin.csv", index=False)


In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
rgrs = RandomForestRegressor(max_depth = 5, random_state=42)

In [ ]:
rgrs.fit(df[['x2', 'x3', 'sex', 'city', 'university', 'age_mean', 'u_v', 'school', 'age_dist']], df['x1'])

RandomForestRegressor(max_depth=5, random_state=42)

In [56]:
pred=rgrs.predict(df[['x2', 'x3', 'sex', 'city', 'university', 'age_mean', 'u_v', 'school', 'age_dist']])

In [57]:
from sklearn.metrics import mean_squared_error
mean_squared_error(df['x1'], pred)

1.0201155539899025

In [60]:
import pickle

In [62]:
# Предположим, что у вас есть объект модели с именем model
# Сохраняем модель на диск
with open('model.pkl', 'wb') as f:
    pickle.dump(rgrs, f)